# Installation des librairies

In [1]:
!pip install pandas googletrans==4.0.0-rc1
!pip install datasets

# Importation des librairies

In [2]:
import csv

from datasets import load_dataset

from googletrans import Translator

import pandas as pd

/Users/nicolasmelaerts/.pyenv/versions/3.10.1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Chargement d'un dataset de fake-news

In [3]:
ds = load_dataset("AlexanderHolmes0/true-fake-news-v2")

# Initialisation d'un translator et fonction de traduction

In [4]:
# Initialiser le traducteur
translator = Translator()

# Fonction de traduction
def translate(text):
    try:
        return translator.translate(text, src='en', dest='fr').text
    except Exception as e:
        # En cas d'erreur, on affiche le message d'erreur et on retourne None pour ne pas ajouter de texte
        print(f"Erreur de traduction : {e}")
        return None

# Fonction pour continuer la truduction des news du dataset

In [5]:
def continue_translation(output_filename, dataset_name, num_to_translate):
    ds = load_dataset(dataset_name)['train']

    # Vérifier le nombre de lignes déjà traduites
    try:
        with open(output_filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            existing_lines = list(reader)
        translated_count = len(existing_lines)
    except FileNotFoundError:
        print(f"Fichier {output_filename} non trouvé. Un nouveau fichier sera créé.")
        translated_count = 0

    # Continuer la traduction
    csv_data = []
    for i in range(translated_count, min(translated_count + num_to_translate, len(ds))):
        entry = ds[i]
        data = translate(entry['text'])
        if data:  # Vérifie que la traduction a réussi
            label = entry['label']
            csv_data.append({
                'empty_column': '',
                'data': data,
                'label': 1 if label == 1 else 0
            })

    # Ajouter les news dans le fichier
    with open(output_filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['empty_column', 'data', 'label'])
        if translated_count == 0:  # Écrire l'en-tête seulement si le fichier est nouveau
            writer.writeheader()
        writer.writerows(csv_data)

    print(f"Traduction ajoutée avec succès dans {output_filename}. Total de lignes : {translated_count + len(csv_data)}.")

# Fonction pour vérifier qu'il n'y a pas de case vide dans le fichier

In [6]:
def clean_csv(input_filename, output_filename):
    cleaned_data = []
    try:
        with open(input_filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Vérifier si les champs 'data' ou 'label' sont vides
                if row['data'].strip() and row['label'].strip():
                    cleaned_data.append(row)
    except FileNotFoundError:
        print(f"Le fichier {input_filename} n'existe pas.")
        return

    with open(output_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['empty_column', 'data', 'label'])
        writer.writeheader()
        writer.writerows(cleaned_data)

    print(f"Fichier nettoyé créé avec succès : {output_filename}")

# Fonction pour compter le nombre de news dans un fichier

In [7]:
def count_data_in_csv(filename):
    try:
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            data_count = sum(1 for _ in reader)
        print(f"Nombre total de données dans le fichier {filename} : {data_count}")
        return data_count
    except FileNotFoundError:
        print(f"Le fichier {filename} n'existe pas.")
        return 0

In [8]:
continue_translation('augmented.csv', "AlexanderHolmes0/true-fake-news-v2", 1)
clean_csv('augmented.csv', 'translated_news_cleaned.csv')
count_data_in_csv('translated_news_cleaned.csv')

Traduction ajoutée avec succès dans augmented.csv. Total de lignes : 10002.
Fichier nettoyé créé avec succès : translated_news_cleaned.csv
Nombre total de données dans le fichier translated_news_cleaned.csv : 10002


10002

# Fonction pour ajouter le dataset train fourni et encore augmenter les datas

In [9]:
# Charger les fichiers
fake_train = pd.read_csv("fake_train.csv")
translated_news_cleaned = pd.read_csv("translated_news_cleaned.csv")

# Vérifier les doublons et ajouter uniquement les nouvelles lignes
combined = pd.concat([translated_news_cleaned, fake_train]).drop_duplicates(subset=['data', 'label']).reset_index(drop=True)

# Sauvegarder le fichier mis à jour
combined.to_csv("translated_news_cleaned_with_train.csv", index=False)

print(f"Les données ont été combinées et sauvegardées dans 'translated_news_cleaned_with_train.csv'.")


Les données ont été combinées et sauvegardées dans 'translated_news_cleaned_with_train.csv'.


# Fonction pour compter le nombre de fake news et de true news dans un csv

In [10]:
# Chemin vers votre fichier CSV
file_path = 'translated_news_cleaned_with_train.csv'

# Initialisation des compteurs pour 0 et 1
count_0 = 0
count_1 = 0

# Ouverture et lecture du fichier CSV
with open(file_path, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    # Parcours des lignes du fichier CSV
    for row in reader:
        # Vérifie la valeur de la colonne 'label' et incrémente le compteur correspondant
        if row['label'] == '0':
            count_0 += 1
        elif row['label'] == '1':
            count_1 += 1

# Affichage des résultats
print(f"Nombre de 0 dans la colonne 'label' : {count_0}")
print(f"Nombre de 1 dans la colonne 'label' : {count_1}")


Nombre de 0 dans la colonne 'label' : 5161
Nombre de 1 dans la colonne 'label' : 5174
